In [1]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 7.9 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp39-cp39-linux_x86_64.whl size=661963 sha256=28b2ded2ebe34f3b70c1e0ddbb0b79be1df2bccf5cf1c2d2fd0de4b60687b2cb
  Stored in directory: /root/.cache/pip/wheels/82/09/27/74d8e31ed19c530166e0d263aabe1ea57465e255615bda8fc0
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69866 sha256=c41c3c98da2e8351e81c4e6994e2adacb860514a0d16135f36028fe88d62f20b
  Stored in directory: /root/.cache/pip/wheels/76/ee/8c

In [3]:
#vigenere cipher serially
from time import time
import re
def generateKey(string, key):
    key = list(key)
    if len(string) == len(key):
        return(key)
    else:
        for i in range(len(string) -
                       len(key)):
            key.append(key[i % len(key)])
    return("" . join(key))

def Encrypt(plaintext, key):
    cipher_text = []
    for i in range(len(plaintext)):
        x = ((ord(plaintext[i]) - ord('A')) + (ord(key[i]) - ord('A'))) % 26
        # = (Ascii plaintext charactor - 65) + (Ascii key charactor - 65) % 26
        x += ord('A')
        # = number + 65
        cipher_text.append(chr(x))
    
    return("" . join(cipher_text))

def Decrypt(cipher_text, key):
    orig_text = []
    for i in range(len(cipher_text)):
        x = ((ord(cipher_text[i]) - ord('A')) - (ord(key[i]) - ord('A'))) % 26
        # = (Ascii plaintext charactor - 65) - (Ascii key charactor - 65) % 26
        x += ord('A')
        # = number + 65
        orig_text.append(chr(x))

    return("" . join(orig_text))


file = open('25000 words.txt', 'r')
plaintext = file.read()
plaintext=re.sub("[^A-Za-z]","",plaintext)

keyword = "strike"
plaintext = plaintext.replace(" ","").upper()
key = generateKey(plaintext, keyword).upper()
#original = plaintext

start_time = time()

cipher_text = Encrypt(plaintext,key)
decrypted = Decrypt(cipher_text, key)
print(round(time() - start_time,8), 'seconds')


# print("Ciphertext :", cipher_text)

#print(decrypted == original)

# print("Decrypted Text :",decrypted)


FileNotFoundError: ignored

In [ ]:
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit  
import math 
import numpy
import re
from time import time


def generateKey(string, key):
    key = list(key)
    if len(string) == len(key):
        return(key)
    else:
        for i in range(len(string) -
                       len(key)):
            key.append(key[i % len(key)])
    return("" . join(key))

def to_char(text):
    ascii_values = [chr(character) for character in text]
    return ascii_values
def to_ascii(text):
    ascii_values = [ord(character) for character in text]
    return ascii_values

mod = SourceModule("""
    __global__ void vigenereCipherEnc(int *plaintext,int *key,int n_blocks,int Maxsize)
    {
        int x =  blockIdx.x * blockDim.x + threadIdx.x ;
        int y =  blockIdx.y * blockDim.y + threadIdx.y;
        int idx = (gridDim.x * blockDim.x) * y + x;

        for( int i=n_blocks*idx ; i<n_blocks*(idx+1)&& i<Maxsize ;i++)
        {
          plaintext[i] = (plaintext[i] - 65) + (key[i] - 65);
          plaintext[i] = plaintext[i] % 26;
          plaintext[i] = plaintext[i] + 65;
        }
        
    }
    __global__ void vigenereCipherDec(int *cipher,int *key,int n_blocks,int Maxsize)
    {
        int x =  blockIdx.x * blockDim.x + threadIdx.x ;
        int y =  blockIdx.y * blockDim.y + threadIdx.y;
        int idx = (gridDim.x * blockDim.x) * y + x;

        for( int i=n_blocks*idx ; i<n_blocks*(idx+1)&& i<Maxsize ;i++)
        {
          cipher[i] = (cipher[i] - 65) - (key[i] - 65);
          cipher[i] = cipher[i] %26;
          if(cipher[i] <0)
          {
            cipher[i] = cipher[i] + 26;
          }
          cipher[i] = cipher[i] +65;
        }
    }
""")


file = open('25000 words.txt', 'r')
plaintext = file.read()
plaintext=re.sub("[^A-Za-z]","",plaintext)
keyword = "strike".upper()

plaintext = plaintext.replace(" ","").upper()
key = generateKey(plaintext, keyword)
original = plaintext


plaintext = numpy.array(to_ascii(plaintext.upper()),dtype=numpy.int32)
key = numpy.array(to_ascii(key.upper()),dtype=numpy.int32)

threads = 32
n_blocks = math.ceil(len(plaintext)/threads)

text_gpu = cuda.to_device(plaintext)
key_gpu = cuda.to_device(key)

enc = mod.get_function("vigenereCipherEnc")
desc = mod.get_function("vigenereCipherDec")

start_time = time()

enc(text_gpu,key_gpu,numpy.uint32(n_blocks),numpy.uint32(len(plaintext)), block=(threads, 1, 1))
cipher_text = cuda.from_device_like(devptr=text_gpu, other_ary=plaintext)  

desc(text_gpu,key_gpu,numpy.uint32(n_blocks),numpy.uint32(len(plaintext)), block=(threads, 1, 1))
d = cuda.from_device_like(devptr=text_gpu, other_ary=plaintext)  

decrypted = "".join(to_char(d))

print(round(time() - start_time,8), 'seconds')

# print("Original  :",original)
# print("Cipher    :","".join(to_char(cipher_text)))
# print("Decrypted :","".join(to_char(d)))
print(decrypted == original)

In [ ]:
import matplotlib.pyplot as plt
time = [5.20515,10.16355,14.46533,20.40815,25.42353]
words = [5000,10000,15000,20000,25000]
plt.plot(words,time,color="g",marker=".",label="32 threads")

plt.xlabel('Time')
plt.ylabel('word size')
plt.legend()
plt.show()


In [ ]:
# while True:
#   pass